First, let's import all the necessary dependencies.

In [ ]:
from cartopy import crs, feature
import display
import matplotlib.pyplot as plt
import xarray as xr

Let's define some configuration variables.

In [ ]:
AREA = {"latitude": slice(55, 45), "longitude": slice(2, 24)}
DATASET = "s3://hedp/era5/ecv-for-climate-change-1979-2023.zarr"
PROJECTION = crs.Miller()
SEASON = "12"

In the following lines we:
- establish a connection with the remote dataset
- show its metadata

It is worth noticing that, in the first operation, only the information necessary to gather the dataset's metadata (e.g. its sizes, dimensions, variables' type and attributes) is actually downloaded from its remote location to the local machine.

In [ ]:
ds = xr.open_dataset(DATASET, chunks={}, engine="zarr").astype("float32")
ds

Here we want to compute the average anomaly of the 2-metre Temperature in Winter 2022 over Central Europe.

Note that the operations prescribed in the following lines are actually performed, and the resulting data downloaded, only when the `.compute()` method is met. For this reason, as a general rule to maximize efficiency, the `compute()` method should be placed after an operation which significantly reduce the amount of data, such as after selection and reduction operations.

In [ ]:
t2m = ds.t2m
t2m_area = t2m.sel(**AREA)
t2m_area_seasonal_mean = t2m_area.resample(time="QS-DEC").mean(dim="time")
t2m_area_season_mean = t2m_area_seasonal_mean.sel(time=f"2022-{SEASON}")
t2m_area_season_mean

In [ ]:
%%time
%%capture

t2m_area_season_mean = t2m_area_season_mean.compute()

Let's plot the results on a map.

In [ ]:
_, ax = plt.subplots(
    figsize=(6, 6),
    subplot_kw={"projection": PROJECTION},
)
t2m_area_season_mean.plot(
    ax=ax, 
    cmap="YlOrRd", 
    transform=crs.PlateCarree(), 
    cbar_kwargs={"orientation": "horizontal", "pad": 0.05, "aspect": 40, "label": "Surface Temperature anomaly [C]"},
)
ax.coastlines()
ax.add_feature(feature.BORDERS)
ax.set_title("Surface Temperature anomaly - Winter 2022")
plt.show()

Now, we want to compute the average 2-metre Temperature anomaly over Central Europe for each Winter in the available time range.

The same considerations done before apply here.

In [ ]:
t2m = ds.t2m
t2m_area = t2m.sel(**AREA)
t2m_area_seasonal_spatial_mean = t2m_area.resample(time="QS-DEC").mean(dim=["time", "latitude", "longitude"])
t2m_area_seasonal_spatial_mean

In [ ]:
%%time

t2m_area_seasonal_spatial_mean = t2m_area_seasonal_spatial_mean.compute()
t2m_area_winter_mean = t2m_area_seasonal_spatial_mean.sel(time=t2m_area_seasonal_spatial_mean["time.month"] == int(SEASON))

Finally, let's plot the results on a time series.

In [ ]:
plt.plot(t2m_area_winter_mean["time"], t2m_area_winter_mean)
plt.xlabel("Year")
plt.ylabel("Surface Temperature anomaly [C]")
plt.title("Mean Winter surface temperature anomaly in Central Europe", size=10)
plt.show()